---
title: EpigenomeXcan Test
date: 12/27/2023
author: Sabrina Mi
---


In [1]:
import numpy as np
import h5py
import tensorflow as tf
import sqlite3
import pandas as pd

2024-01-01 00:07:28.645287: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE3 SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-01 00:07:32.645283: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
gene = "ENSRNOG00000070168"
chr = "chr1"
tss = 157231467
start = tss-57344
end = tss+57344
sample = "00078A02B6"

In [3]:
columns = ['POS', 'ACI', 'BN', 'BUF', 'F344', 'M520', 'MR', 'WKY', 'WN']
def query(cursor, sample, start, end):
    # Execute SQL query
    cursor.execute(f'SELECT * FROM `{sample}` WHERE POS BETWEEN {start} AND {end-1}')
    # Fetch all rows
    data = cursor.fetchall()
    df = pd.DataFrame(data, columns=columns)
    return df

In [4]:
conn = sqlite3.connect(f"/home/s1mi/Br_genotype_probabilities/{chr}_probabilities.db")
cursor = conn.cursor()
prob = query(cursor, sample, start, end)

In [5]:
pr_mean = prob.mean()[1:]
pr_matrix = np.tile(pr_mean, (896,1))
indices = (prob['POS'] - start) // 128
pr_matrix[indices, :] = prob.iloc[:, 1:].values

In [6]:
with h5py.File(f"/eagle/AIHPC4Edu/sabrina/Br_predictions/HS_founder_epigenomes/human/{chr}_genes.h5", "r") as hf:
    matrix = hf[gene][:]


In [13]:
ref_matrix = np.transpose(matrix, axes=(1, 0, 2))

In [27]:
res_matrix = np.squeeze(pr_matrix[:, np.newaxis,:] @ ref_matrix, 1)